# Reestruturação MVC com Heurísticas de Nielsen

Este notebook segue o padrão **MVC** (Model-View-Controller), organizando as funções para maior clareza e modularidade.  
Também aplicamos duas heurísticas de Nielsen:
- **Visibilidade do estado do sistema**: Mensagens informam o progresso em cada etapa.
- **Consistência e padrões**: Estrutura modular com células dedicadas para cada componente.

## Estrutura
1. **Model**: Gerenciamento de dados e lógica de previsão.
2. **View**: Apresentação de resultados, gráficos e outputs visuais.
3. **Controller**: Interação entre Model e View, coordenando inputs e outputs.

<h3>Model: Gerenciamento de Dados e Lógica</h3>

In [1]:
import pandas as pd

# Função para carregar os dados
def carregar_dados(caminho_arquivo):
    return pd.read_csv(caminho_arquivo)

# Função para processar dados de veículos elétricos
def processar_dados_veiculos(dados):
    # Se necessário, adicionar mais processamento dos dados de veículos
    return dados

# Função para processar dados de localização (Latitude e Longitude)
def processar_dados_localizacao(dados, lat_col, long_col):
    dados = dados.dropna(subset=[lat_col, long_col])  # Remove NaN na localização
    return dados

# Função para processar os dados de estações de carregamento
def processar_dados_estacoes(dados):
    dados.columns = dados.columns.str.strip()  # Remover espaços em branco nos nomes das colunas
    return dados

# Função para limpar dados de carregadores (remover NaNs nas colunas kwhTotal e dollars)
def limpar_dados_carregadores(dados):
    dados_limpos = dados.dropna(subset=['kwhTotal', 'dollars'])  # Remover NaN em 'kwhTotal' e 'dollars'
    return dados_limpos

# Função para calcular a correlação entre veículos e carregadores
def calcular_correlacao(cidade_carros, cidade_station):
    merged_data = pd.merge(cidade_carros, cidade_station, on='City', how='inner')
    return merged_data['Número de Veículos'].corr(merged_data['Número de Chargers'])


<h3>View: Apresentação dos Resultados</h3>

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

# Função para exibir o gráfico das 50 cidades com mais veículos elétricos
def exibir_top_50_cidades_com_mais_veiculos(dados):
    cidade_carros = dados['City'].value_counts().reset_index()
    cidade_carros.columns = ['City', 'Número de Veículos']
    top_50_cidades = cidade_carros.head(50)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(data=top_50_cidades, x='City', y='Número de Veículos')
    plt.title('Top 50 Cidades com Mais Veículos Elétricos', fontsize=14)
    plt.xlabel('Cidade', fontsize=12)
    plt.ylabel('Número de Veículos', fontsize=12)
    plt.xticks(rotation=90)
    plt.grid(True)
    plt.show()

# Função para exibir gráfico de dispersão entre kWh e preço
def exibir_grafico_dispersao(dados):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=dados, x='kwhTotal', y='dollars', alpha=0.6)
    plt.title('Relação entre Demanda (kWh) e Preço ($)', fontsize=14)
    plt.xlabel('Demanda (kWh)', fontsize=12)
    plt.ylabel('Preço ($)', fontsize=12)
    plt.grid(True)
    plt.show()

# Função para exibir a correlação entre veículos e carregadores
def exibir_correlacao_veiculos_carregadores(correlacao):
    print(f"Correlação entre número de veículos e carregadores: {correlacao:.2f}")


<h3>Controller: Fluxo de Execução</h3>

In [3]:
# Controller: Executa o pipeline de análise e visualização
if __name__ == "__main__":
    # Caminhos dos arquivos
    caminho_ve = "Electric_Vehicle_Population_Data.csv"
    caminho_estacoes = "Electric_Vehicle_Charging_Stations.csv"
    caminho_estacoes_dv = "station_data_dataverse.csv"

    # Carregar os datasets (Model)
    dados_ve = carregar_dados(caminho_ve)
    dados_estacoes = carregar_dados(caminho_estacoes)
    dados_estacoes_dv = carregar_dados(caminho_estacoes_dv)

    # Processar os dados de veículos elétricos (Model)
    dados_ve = processar_dados_veiculos(dados_ve)
    dados_ve = processar_dados_localizacao(dados_ve, "Latitude", "Longitude")

    # Processar os dados de estações de carregamento (Model)
    dados_estacoes = processar_dados_estacoes(dados_estacoes)

    # Limpar dados de carregadores (Model)
    dados_estacoes_limpos = limpar_dados_carregadores(dados_estacoes)

    # Visualizações (View)
    exibir_top_50_cidades_com_mais_veiculos(dados_ve)  # Top 50 cidades com mais carros elétricos
    exibir_grafico_dispersao(dados_estacoes_limpos)  # Gráfico de relação entre kWh e preço

    # Calcular a correlação entre veículos e carregadores (Model)
    cidade_carros = dados_ve['City'].value_counts().reset_index()
    cidade_carros.columns = ['City', 'Número de Veículos']
    cidade_station = dados_estacoes['City'].value_counts().reset_index()
    cidade_station.columns = ['City', 'Número de Chargers']
    
    correlacao = calcular_correlacao(cidade_carros, cidade_station)  # Calcular correlação entre veículos e carregadores

    # Exibir correlação (View)
    exibir_correlacao_veiculos_carregadores(correlacao)


KeyError: ['Latitude', 'Longitude']